In [41]:
%matplotlib inline

from pathlib import Path

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
import matplotlib.pylab as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from mord import LogisticIT
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier, KNeighborsRegressor
from dmba import regressionSummary, classificationSummary, liftChart, gainsChart
import numpy as np
import seaborn as sns
DATA = Path('C:\\Users\\tanve\\Documents\\206\\dmba\\')

In [42]:
sba_df = pd.read_csv(DATA / 'SBAcase.11.13.17.csv')
Subset = [
       'DisbursementGross','New', 'RealEstate', 'Portion', 'Recession', 'Default'
       ]
outcome = 'Default'

#### The variable “Selected” indicates which observations are the “training” data and which are the “testing” data (1 = training data to be used to build the model, 0 = testing data to validate the model). Partition the data using this variable.

In [43]:
train_df = sba_df[sba_df["Selected"]== 1]
valid_df = sba_df[sba_df["Selected"]== 0]
train_df = train_df[Subset]
valid_df = valid_df[Subset]


###### using the subsets from the initial sba work to reduce redundency 

#### (a) Build a logistic regression model using STATMODELS sm.glm or smf.glm in Python to reproduce results (not format) in Tables 7(a), 8, 9 of this article using the SBA case data SBAcase.11.13.17.csv.

In [44]:
train_sba_df2 = sm.add_constant(train_df, prepend=True)
train_sba_df2.columns = [s.strip().replace('/', '_') for s in train_sba_df2.columns]
valid_sba_df2 = sm.add_constant(valid_df, prepend=True)
valid_sba_df2.columns = [s.strip().replace('/', '_') for s in valid_sba_df2.columns]

In [45]:
#for pred in Subset:
#    train_sba_df2[pred] = train_sba_df2[pred].astype('category')
#train_sba_df2.Default = train_sba_df2['Default'].astype('category')
# Results using the entire data for comparison with sklearn or R
train_y2 = train_sba_df2[outcome]
#train_X2 =  pd.get_dummies(train_sba_df2.drop(columns=outcome),prefix_sep='_', drop_first=False)
train_X2 = train_sba_df2.drop(columns=outcome)
valid_y2 = valid_sba_df2[outcome]
valid_X2 = valid_sba_df2.drop(columns=outcome)

In [46]:
logit_reg2 = sm.GLM(train_y2, train_X2, family=sm.families.Binomial())
logit_result2 = logit_reg2.fit()
logit_result2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                Default   No. Observations:                 1051
Model:                            GLM   Df Residuals:                     1045
Model Family:                Binomial   Df Model:                            5
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -540.85
Date:                Mon, 25 Sep 2023   Deviance:                       1081.7
Time:                        16:06:01   Pearson chi2:                 1.00e+03
No. Iterations:                     6   Pseudo R-squ. (CS):             0.1949
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 1.3537      0.323      4.192      0.000       0.721       1.987
DisbursementGross -3.371e-07   3.52e-07     -0.958      0.338   -1.03e-06    3.53e-07
New                  -0.0772      0.210     -0.367      0.713      -0.489       0.335
RealEstate           -2.0331      0.364     -5.592      0.000      -2.746      -1.320
Portion              -2.8298      0.559     -5.059      0.000      -3.926      -1.733
Recession             0.4971      0.241      2.060      0.039       0.024       0.970
=====================================================================================
"""

#### (b) Explain why risk indicators in Table 8 were selected using p-values in Table 7(a).

The three slected risk indicators used in table 8 were the risk indicators in table 7(a) that were the closest in p-value to the intercept. 

#### c) Interpret parameter (coefficient) estimates of the model in Table 8 with a focus on the odds of default. Answer the following questions by interpreting parameter estimates of the model in Table 8 and specifying odds and probabilities of default for these risk indicators.

##### (i) Is a loan backed by real estate more likely or less likely to default (by how much)? Explain using parameter estimates.

the negative coeff value of RealEstate tells us that a loan is less likly to deafult if backed by real esate. It looks like they dont default at twice the rate as others default. so it would be a 2:1 ratio. So a 67% likelyhood of paying it off or a ~33% reduction in defaulting. 

##### (ii) Is a loan active during recession more likely or less likely to default (by how much)? Explain using parameter estimates.

The postive coeff value of Recession tells us that a loan is more likely to default during a recession. The effect of a recession is smaller

#### (iii) How much does the portion of a loan guaranteed by SBA increase or decrease the likelihood of default? Explain using parameter estimates

It decreases the likelihood of default by a factor of 2.83.

### (d) For the California-based example, the final model with the risk indicators in Table 8 is used to estimate the probability of default for the two loan applications. Use Python to predict the probability of default for Carmichael Realty (Loan 1) and SV Consulting (Loan 2). Applying the decision rules and cutoff probability of 0.5 from Section 4.3Links to an external site., how should these two loans be classified as, lower risk (approve) or higher risk (deny)?

****** the mentioned records dont exist in the dataframe, so i conducted analysis on the two loan applications from Carmicael. **********

In [62]:
cram_df = sba_df[sba_df["City"]== "CARMICHAEL"]
cram_df

,Selected,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,...,ChgOffPrinGr,GrAppv,SBA_Appv,New,RealEstate,Portion,Recession,daysterm,xx,Default
873,1,2894155009,"ABC REALTY & MORTGAGES, INC.",CARMICHAEL,CA,95608,U.S. BANK NATIONAL ASSOCIATION,OH,531210,17538,...,0,20000,10000,0,0,0.5,0,2520,20537.0,0
1845,1,7958734002,NOONIS DEVELOPMENT INC,CARMICHAEL,CA,95608,CAPITAL ONE NATL ASSOC,VA,531390,16363,...,0,35000,17500,0,0,0.5,0,2520,18956.0,0


In [ ]:
cram_df = cram_df[Subset]
cram_df2 = sm.add_constant(cram_df, prepend=True, has_constant='add')
cram_df2.columns = [s.strip().replace('/', '_') for s in cram_df2.columns]
cram_valid_X2 = cram_df2.drop(columns=outcome)
cram_valid_y2 = cram_df2[outcome]

In [59]:
print(valid_X2.shape)
print(cram_valid_X2.shape)

(1051, 6)
(2, 6)


In [60]:
logit = sm.GLM(train_y2, train_X2, family=sm.families.Binomial())

result = logit.fit()

predictions = result.predict(cram_valid_X2)

predictions_nominal = [ 0 if x < 0.5 else 1 for x in predictions]

logit_result = pd.DataFrame({'actual': cram_valid_y2, 'p(0)': 1 - predictions,

                             'p(1)': predictions,

                             'predicted': predictions_nominal })

logit_result

,actual,p(0),p(1),predicted
873,0,0.519151,0.480849,0
1845,0,0.518226,0.481774,0


Using a cutoff value of 0.5, both of the observed loans are classified as high liklihood of not defaulting. This would make them low risk and likely to be approved. 